# Multi-Financial Agent Notebook
This notebook demonstrates the implementation of a multi-financial agent system. It includes the association of supervisor agents with sub-agents and other related operations.

### Associating Supervisor Agent with Sub-Agents
The following code associates a supervisor agent with a list of sub-agents. This is done using the `associate_sub_agents` method from the `agents` module.  
- `supervisor_agent_id`: The unique identifier for the supervisor agent.  
- `sub_agents_list`: A list of sub-agent IDs to be associated with the supervisor agent.  
- The method returns:  
  - `supervisor_agent_alias_id`: The alias ID for the supervisor agent.  
  - `supervisor_agent_alias_arn`: The Amazon Resource Name (ARN) for the supervisor agent.

## Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

In [ ]:
!pip uninstall boto3 botocore awscli --yes


In [ ]:
# Install latest boto3

In [ ]:
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt


## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and you will start to create your Supervisor agent.

In [ ]:
import os
import sys
import json
import time
import uuid
import boto3
import logging

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from globals import *

### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:

- `create_agent`: Create a new agent and respective IAM roles
- `associate_sub_agents`: Associate sub-agents with multi-agent collaborator

In [ ]:

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### let us set the sessions right before we start looping in the agents.

In [ ]:
session = boto3.session.Session()
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

logger.info(f"current region: {region}")

In [ ]:

data_assistant_agent_alias_arn = '#set the arn of the forecasting agent'


portfolio_agent_alias_arn = '#set the arn of portfolio agent'

## Agent instruction 

Now that we have the agents assigned to the variables, we need to go and add the agent instructions for the main agent here

In [ ]:
agent_instruction: str = """You are an investment analyst. Your job is to assist in investment analysis, create research summaries, generate profitable company portfolios, and facilitate communication through emails. Here is how I want you to think step by step:

1. Portfolio Creation:
    Analyze the user's request to extract key information such as the desired number of companies and industry. 
    Based on the criteria from the request, create a portfolio of companies. Use the template provided to format the portfolio.

2. Company Research and Document Summarization:
    For each company in the portfolio, conduct detailed research to gather relevant financial and operational data.
    When a document, like the FOMC report, is mentioned, retrieve the document and provide a concise summary.

3. Email Communication:
    Using the email template provided, format an email that includes the newly created company portfolio and any summaries of important documents.
    Utilize the provided tools to send an email upon request, That includes a summary of provided responses and portfolios created.
""

## Adding Sub Agents. 

The `sub_agents_list` will be used to encapsulate the agent alias and instructions for the multi_agent orchestrator to add them later on. 

In [ ]:
agent_name = 'Multiagent analysis'
agent_description = "Multi-agent collaboration for getting data and Portfolio management"

sub_agents_list = [
    {
        'sub_agent_alias_arn': data_assistant_agent_alias_arn,
        'sub_agent_instruction': """Use this agent when the user asks a question that requires to retrieve data that the user asks related to FOMC reports or other things""",
        'sub_agent_association_name': 'data_assistant_agent-agent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': portfolio_agent_alias_arn,
        'sub_agent_instruction': """Use this agent when a user asks to assist in investment analysis, create research summaries, generate profitable company portfolios, and facilitate communication through emails""",
        'sub_agent_association_name': 'portfolio_agent-agent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

In [2]:
## Creating Supervisor Agent

In [ ]:
if region == "us-west-2":
    NOVA_LITE = f"us.{NOVA_LITE}"
    logger.info(f"Current region is {region}. Using the cross region inference profile model id: {NOVA_LITE}")

supervisor_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    NOVA_LITE,
    agent_collaboration='SUPERVISOR_ROUTER'
)
supervisor_agent

In [ ]:
supervisor_agent_id = supervisor_agent[0]
%store supervisor_agent_id
%store agent_name

## add the subagents to the supervisor agents

In [ ]:
supervisor_agent_alias_id, supervisor_agent_alias_arn = agents.associate_sub_agents(
    supervisor_agent_id, sub_agents_list
)

### Next Steps
Congratulations! We've now created a supervisor agent. Next let's invoke our agent

#### Multiple agents in parallel

Finally, let's submit a query which will result in our supervisor agent requiring responses from multiple agents in parallel. This shows the behaviour of a supervisor agent using supervisor mode rather than router mode as we have seen in the previous examples.

In [ ]:
multi_agent_names = {
    f'{data_assistant_agent_id}/{data_assistant_agent_alias_id}': SUB_AGENT_DATA_ASSISTANT_AGENT,
    f'{portfolio_agent_agent_id}/{portfolio_agent_agent_alias_id}': SUB_AGENT_NAME_PORTFOLIO_AGENT,
    
}
multi_agent_names
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    """Give me details about the Technologies companies""", 
    supervisor_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

## Congratulations

By now you have created 2 sub-agents and a supervisor agent. You have invoked the supervisor agent using prompts requiring multiple sub-agents.

For other multi-agent collaboration examples check the [Amazon Bedrock Agent Samples](https://github.com/awslabs/amazon-bedrock-agent-samples) repository

Next, let's clean up our resources to avoid unexpected costs
